In [1]:
%matplotlib inline

import numpy as np
import random
import requests as rq
import sys
import io
import re
import pandas as pd
from gensim import models
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = WordNetLemmatizer()

/Users/abby/Applications/miniconda3/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package punkt to /Users/abby/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/abby/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/abby/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
model_fox = Word2Vec.load("./models/fox.model")
model_reuters = Word2Vec.load("./models/reuters.model")
model_the_hill = Word2Vec.load("./models/the_hill.model")
model_cnn = Word2Vec.load("./models/cnn.model")
model_nyt = Word2Vec.load("./models/nyt.model")

In [3]:
adjectives = ['disorganized', 'devious', 'impressionable', 'circumspect', 'impassive', 
             'aimless', 'effeminate', 'unfathomable', 'fickle', 'unprincipled', 'inoffensive', 
             'reactive', 'providential', 'resentful', 'bizarre', 'impractical',
             'sarcastic', 'misguided', 'imitative', 'pedantic', 'venomous', 'erratic', 'insecure', 
             'resourceful', 'neurotic', 'forgiving', 'profligate', 'whimsical', 'assertive', 
             'incorruptible', 'individualistic', 'faithless', 'disconcerting', 'barbaric', 
             'hypnotic', 'vindictive', 'observant', 'dissolute', 'frightening', 'complacent', 
             'boisterous', 'pretentious', 'disobedient', 'tasteless', 'sedentary', 
             'sophisticated', 'regimental', 'mellow', 'deceitful', 'impulsive', 'playful', 
             'sociable', 'methodical', 'willful', 'idealistic', 'boyish', 'callous', 'pompous', 
             'unchanging', 'crafty', 'punctual', 'compassionate', 'intolerant', 'challenging', 
             'scornful', 'possessive', 'conceited', 'imprudent', 'dutiful', 'lovable', 
             'disloyal', 'dreamy', 'appreciative', 'forgetful', 'unrestrained', 'forceful', 
             'submissive', 'predatory', 'fanatical', 'illogical', 'tidy', 'aspiring', 'studious', 
             'adaptable', 'conciliatory', 'artful', 'thoughtless', 'deceptive', 'frugal', 
             'reflective', 'insulting', 'unreliable', 'stoic', 'hysterical', 'rustic', 
             'inhibited', 'outspoken', 'unhealthy', 'ascetic', 'skeptical', 'painstaking', 
             'contemplative', 'leisurely', 'sly', 'mannered', 'outrageous', 'lyrical', 
             'placid', 'cynical', 'irresponsible', 'vulnerable', 'arrogant', 'persuasive', 
             'perverse', 'steadfast', 'crisp', 'envious', 'naive', 'greedy', 'presumptuous', 
             'obnoxious', 'irritable', 'dishonest', 'discreet', 'sporting', 'hateful', 
             'ungrateful', 'frivolous', 'reactionary', 'skillful', 'cowardly', 'sordid', 
             'adventurous', 'dogmatic', 'intuitive', 'bland', 'indulgent', 'discontented', 
             'dominating', 'articulate', 'fanciful', 'discouraging', 'treacherous', 
             'repressed', 'moody', 'sensual', 'unfriendly', 'optimistic', 'clumsy', 
             'contemptible', 'focused', 'haughty', 'morbid', 'disorderly', 'considerate', 
             'humorous', 'preoccupied', 'airy', 'impersonal', 'cultured', 'trusting', 
             'respectful', 'scrupulous', 'scholarly', 'superstitious', 'tolerant', 
             'realistic', 'malicious', 'irrational', 'sane', 'colorless', 'masculine', 
             'witty', 'inert', 'prejudiced', 'fraudulent', 'blunt', 'childish', 'brittle', 
             'disciplined', 'responsive', 'courageous', 'bewildered', 'courteous', 
             'stubborn', 'aloof', 'sentimental', 'athletic', 'extravagant', 'brutal', 
             'manly', 'cooperative', 'unstable', 'youthful', 'timid', 'amiable', 'retiring', 
             'fiery', 'confidential', 'relaxed', 'imaginative', 'mystical', 'shrewd', 
             'conscientious', 'monstrous', 'grim', 'questioning', 'lazy', 'dynamic', 
             'gloomy', 'troublesome', 'abrupt', 'eloquent', 'dignified', 'hearty', 'gallant', 
             'benevolent', 'maternal', 'paternal', 'patriotic', 'aggressive', 'competitive', 
             'elegant', 'flexible', 'gracious', 'energetic', 'tough', 'contradictory', 
             'shy', 'careless', 'cautious', 'polished', 'sage', 'tense', 'caring', 
             'suspicious', 'sober', 'neat', 'transparent', 'disturbing', 'passionate', 
             'obedient', 'crazy', 'restrained', 'fearful', 'daring', 'prudent', 'demanding', 
             'impatient', 'cerebral', 'calculating', 'amusing', 'honorable', 'casual',
             'sharing', 'selfish', 'ruined', 'spontaneous', 'admirable', 'conventional', 
             'cheerful', 'solitary', 'upright', 'stiff', 'enthusiastic', 'petty', 'dirty', 
             'subjective', 'heroic', 'stupid', 'modest', 'impressive', 'orderly', 'ambitious', 
             'protective', 'silly', 'alert', 'destructive', 'exciting', 'crude', 'ridiculous', 
             'subtle', 'mature', 'creative', 'coarse', 'passive', 'oppressed', 'accessible', 
             'charming', 'clever', 'decent', 'miserable', 'superficial', 'shallow', 'stern', 
             'winning', 'balanced', 'emotional', 'rigid', 'invisible', 'desperate', 'cruel', 
             'romantic', 'agreeable', 'hurried', 'sympathetic', 'solemn', 'systematic', 
             'vague', 'peaceful', 'humble', 'dull', 'expedient', 'loyal', 'decisive', 
             'arbitrary', 'earnest', 'confident', 'conservative', 'foolish', 'moderate', 
             'helpful', 'delicate', 'gentle', 'dedicated', 'hostile', 'generous', 'reliable', 
             'dramatic', 'precise', 'calm', 'healthy', 'attractive', 'artificial', 
             'progressive', 'odd', 'confused', 'rational', 'brilliant', 'intense', 
             'genuine', 'mistaken', 'driving', 'stable', 'objective', 'sensitive', 
             'neutral', 'strict', 'angry', 'profound', 'smooth', 'ignorant', 'thorough', 
             'logical', 'intelligent', 'extraordinary', 'experimental', 'steady', 
             'formal', 'faithful', 'curious', 'reserved', 'honest', 'busy', 'educated', 
             'liberal', 'friendly', 'efficient', 'sweet', 'surprising', 'mechanical', 
             'clean', 'critical', 'criminal', 'soft', 'proud', 'quiet', 'weak', 'anxious', 
             'solid', 'complex', 'grand', 'warm', 'slow', 'false', 'extreme', 'narrow', 
             'dependent', 'wise', 'organized', 'pure', 'directed', 'dry', 'obvious', 'popular', 
             'capable', 'secure', 'active', 'independent', 'ordinary', 'fixed', 'practical', 
             'serious', 'fair', 'understanding', 'constant', 'cold', 'responsible', 'deep', 
             'religious', 'private', 'simple', 'physical', 'original', 'working', 'strong', 
             'modern', 'determined', 'open', 'political', 'difficult', 'knowledge', 'kind']

In [4]:
testing_words = ["gun", "metoo", "america", "tax", "abortion", "democrat", "republican", "nra", 
                 "trump", "biden", "china", "gay", "lesbian"]

In [17]:
def print_adj_dists(model, news_source, word_list):    
    for word in word_list:
      word_dists = []
      for ind in range(len(adjectives)):
        adj = adjectives[ind]
        try:
          dist = model.wv.distance(word, adj)
          word_dists.append([news_source, word, adj, dist])
        except:
          word_dists.append([news_source, word, adj, None])

      df = pd.DataFrame(word_dists, columns=['source','word', 'adj', 'dist'])

      print(df.shape)

      df_sorted = df.sort_values(by='dist', ascending=True)
      print(df_sorted.head(5))
        
def print_word_dists_each_model(wordlist):
    print_adj_dists(model_fox, "Fox News", wordlist)
    print_adj_dists(model_reuters, "Reuters", wordlist)
    print_adj_dists(model_the_hill, "The Hill", wordlist)
    print_adj_dists(model_nyt, "NYT", wordlist)
    print_adj_dists(model_cnn, "CNN", wordlist)

In [18]:
print_word_dists_each_model(['woman'])

(423, 4)
       source   word             adj      dist
2    Fox News  woman  impressionable  0.448281
92   Fox News  woman           stoic  0.483701
86   Fox News  woman     thoughtless  0.492474
76   Fox News  woman      submissive  0.493639
359  Fox News  woman        faithful  0.506519
(423, 4)
      source   word         adj      dist
6    Reuters  woman  effeminate  0.382414
171  Reuters  woman   masculine  0.408297
217  Reuters  woman    maternal  0.439024
234  Reuters  woman      caring  0.454147
183  Reuters  woman   courteous  0.470623
(423, 4)
       source   word             adj      dist
2    The Hill  woman  impressionable  0.500812
302  The Hill  woman        romantic  0.504851
36   The Hill  woman       observant  0.514832
76   The Hill  woman      submissive  0.528473
181  The Hill  woman      courageous  0.547238
(423, 4)
    source   word             adj      dist
76     NYT  woman      submissive  0.407920
6      NYT  woman      effeminate  0.456955
137    NYT  woma

In [28]:
print_word_dists_each_model(['nra'])

(423, 4)
       source word         adj      dist
70   Fox News  nra    disloyal  0.462707
215  Fox News  nra     gallant  0.482656
43   Fox News  nra   tasteless  0.490622
183  Fox News  nra   courteous  0.500640
40   Fox News  nra  boisterous  0.510536
(423, 4)
      source word          adj      dist
87   Reuters  nra    deceptive  0.496788
409  Reuters  nra    religious  0.509816
64   Reuters  nra     scornful  0.514046
17   Reuters  nra    misguided  0.540923
292  Reuters  nra  superficial  0.541505
(423, 4)
       source word         adj      dist
115  The Hill  nra       crisp  0.525751
124  The Hill  nra    sporting  0.615989
182  The Hill  nra  bewildered  0.628105
92   The Hill  nra       stoic  0.631830
76   The Hill  nra  submissive  0.632756
(423, 4)
    source word           adj      dist
119    NYT  nra  presumptuous  0.414142
125    NYT  nra       hateful  0.435297
9      NYT  nra  unprincipled  0.475774
422    NYT  nra          kind  0.477982
321    NYT  nra       help

In [29]:
print_word_dists_each_model(['metoo'])

(423, 4)
       source   word          adj      dist
131  Fox News  metoo       sordid  0.397127
81   Fox News  metoo     aspiring  0.448535
96   Fox News  metoo    outspoken  0.494659
257  Fox News  metoo  spontaneous  0.505243
75   Fox News  metoo     forceful  0.507450
(423, 4)
      source   word         adj      dist
93   Reuters  metoo  hysterical  0.410461
6    Reuters  metoo  effeminate  0.440745
62   Reuters  metoo  intolerant  0.446195
297  Reuters  metoo   emotional  0.467436
239  Reuters  metoo  disturbing  0.469760
(423, 4)
       source   word          adj      dist
257  The Hill  metoo  spontaneous  0.451610
200  The Hill  metoo  imaginative  0.477864
212  The Hill  metoo     eloquent  0.485439
133  The Hill  metoo     dogmatic  0.486279
93   The Hill  metoo   hysterical  0.489868
(423, 4)
    source   word           adj      dist
93     NYT  metoo    hysterical  0.494415
128    NYT  metoo   reactionary  0.513331
133    NYT  metoo      dogmatic  0.522175
64     NYT  meto

In [31]:
print_word_dists_each_model(['american'])

(423, 4)
       source      word            adj      dist
286  Fox News  american      oppressed  0.430025
143  Fox News  american      repressed  0.470680
203  Fox News  american  conscientious  0.476445
183  Fox News  american      courteous  0.496688
303  Fox News  american      agreeable  0.500179
(423, 4)
      source      word           adj      dist
182  Reuters  american    bewildered  0.555218
64   Reuters  american      scornful  0.565878
0    Reuters  american  disorganized  0.575307
20   Reuters  american      venomous  0.578273
187  Reuters  american      athletic  0.583312
(423, 4)
       source      word             adj      dist
44   The Hill  american       sedentary  0.452374
13   The Hill  american       resentful  0.462171
216  The Hill  american      benevolent  0.483714
399  The Hill  american        ordinary  0.485799
2    The Hill  american  impressionable  0.493461
(423, 4)
    source      word           adj      dist
137    NYT  american  discontented  0.48510